In [ ]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo

NASA Mars News

In [ ]:
# URL of page to be scraped
url1 = 'https://mars.nasa.gov/news/'

In [ ]:
# Retrieve page with the requests module
response1 = requests.get(url1)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response1.text, 'html.parser')

In [ ]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

In [ ]:
# Collect the latest news title and paragraph text

# Save them in variables
news_title = ""
news_p = ""

JPL Mars Space Images - Featured Image

In [ ]:
# Add splinter depencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# URL of page to be scraped
url2 = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url2)

In [ ]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image
for x in range(1, 6):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # Assign the url string to a variable
    featured_image_url = soup.find_all()

    for quote in quotes:
        print('page:', x, '-------------')
        print(quote.text)

    browser.links.find_by_partial_text('Next').click()

In [ ]:
# Quit browser
browser.quit()

Mars Facts

In [ ]:
# URL of page to be scraped
url3 = 'https://space-facts.com/mars/'

In [ ]:
#
tables = pd.read_html(url3)
tables

In [ ]:
type(tables)

In [ ]:
df = tables[0]
df.head()

In [ ]:
# Clean Up 

In [ ]:
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.


In [ ]:
# Use Pandas to convert the data to a HTML table string.
